In [1]:
import numpy as np
import pandas as pd
from nltk import pos_tag

In [2]:
a=pd.read_csv("twitter_train.csv")
a.head()

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)


In [3]:
b=pd.read_csv("twitter_test.csv")
b.head()

,tweet_id,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,569682010270101504,American,NaN,zsalim03,NaN,0,@AmericanAir In car gng to DFW. Pulled over 1h...,NaN,2015-02-22 18:15:50 -0800,Texas,Central Time (US & Canada)
1,569608307184242688,American,NaN,sa_craig,NaN,0,"@AmericanAir after all, the plane didn’t land ...",NaN,2015-02-22 13:22:57 -0800,"College Station, TX",Central Time (US & Canada)
2,567879304593408001,Southwest,NaN,DanaChristos,NaN,1,@SouthwestAir can't believe how many paying cu...,NaN,2015-02-17 18:52:31 -0800,CT,Eastern Time (US & Canada)
3,569757651539660801,US Airways,NaN,rossj987,NaN,0,@USAirways I can legitimately say that I would...,NaN,2015-02-22 23:16:24 -0800,"Washington, D.C.",Eastern Time (US & Canada)
4,569900705852608513,American,NaN,tranpham18,NaN,0,@AmericanAir still no response from AA. great ...,NaN,2015-02-23 08:44:51 -0800,New York City,Eastern Time (US & Canada)


In [4]:
twit_senti=a[["text",'airline_sentiment']]

In [5]:
twit_senti_test=b[["text"]]
twit_senti_test.head()

,text
0,@AmericanAir In car gng to DFW. Pulled over 1h...
1,"@AmericanAir after all, the plane didn’t land ..."
2,@SouthwestAir can't believe how many paying cu...
3,@USAirways I can legitimately say that I would...
4,@AmericanAir still no response from AA. great ...


In [6]:
twit_senti_test.text

0       @AmericanAir In car gng to DFW. Pulled over 1h...
1       @AmericanAir after all, the plane didn’t land ...
2       @SouthwestAir can't believe how many paying cu...
3       @USAirways I can legitimately say that I would...
4       @AmericanAir still no response from AA. great ...
                              ...                        
3655    @USAirways Been stuck for 40+ minutes due to l...
3656    @USAirways 4 hours... 4 hours... FOUR HOURS.  ...
3657    Nice RT @VirginAmerica: The man of steel might...
3658    @AmericanAir Aww Thanks AA..DFW was on GMA up ...
3659    @united the lounge tells us they have no pillo...
Name: text, Length: 3660, dtype: object

In [7]:
twit_senti

,text,airline_sentiment
0,"@SouthwestAir I am scheduled for the morning, ...",negative
1,@SouthwestAir seeing your workers time in and ...,positive
2,@united Flew ORD to Miami and back and had gr...,positive
3,@SouthwestAir @dultch97 that's horse radish 😤🐴,negative
4,@united so our flight into ORD was delayed bec...,negative
...,...,...
10975,@AmericanAir followback,neutral
10976,@united thanks for the help. Wish the phone re...,positive
10977,@usairways the. Worst. Ever. #dca #customerser...,negative
10978,@nrhodes85: look! Another apology. DO NOT FLY ...,negative


In [8]:
from nltk.stem import WordNetLemmatizer
lematizer=WordNetLemmatizer()

In [9]:
# pos_tag has different part of speech than lammatizer
# we can resolve it with simple fuction
from nltk.corpus import wordnet

def get_simple_pos(tag):
    
    if(tag.startswith('J')):
        return wordnet.ADJ
    elif tag.startswith("V"):
        return wordnet.VERB
    elif tag.startswith("N"):
        return wordnet.NOUN
    elif tag.startswith("R"):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [10]:
from nltk.corpus import stopwords
stops=set(stopwords.words("english"))
import string
punctuations=list(string.punctuation)
stops.update(punctuations)

In [11]:

def clean_review(words):
    output_words=[]
    words=words.split(" ")
    for w in words:
        if w.lower() not in stops and w!='':  # we choosing lower as stop word has only lower stopword
            pos=pos_tag([w])  # we donot choose lower because we may loss some information on lowering it
            clean_word=lematizer.lemmatize(w,pos=get_simple_pos(pos[0][1]))
            output_words.append((clean_word.lower().split('.')[0]).split(',')[0])# we lower because it couldnot differentiate between same word with only difference in large or small
    return output_words

In [12]:
x_train=[clean_review(document) for document in twit_senti.text]

In [13]:
x_train[0]

['@southwestair',
 'schedule',
 'morning',
 '2',
 'day',
 'fact',
 'yes',
 'sure',
 'even',
 'flight',
 'one',
 'cancelled',
 'flightled']

In [14]:
x_test=[clean_review(document) for document in twit_senti_test.text]

In [15]:
y_train=twit_senti.airline_sentiment
y_train

0        negative
1        positive
2        positive
3        negative
4        negative
           ...   
10975     neutral
10976    positive
10977    negative
10978    negative
10979    negative
Name: airline_sentiment, Length: 10980, dtype: object

In [16]:
x_train1=[' '.join(document) for document in x_train]

In [17]:
x_train1[0]

'@southwestair schedule morning 2 day fact yes sure even flight one cancelled flightled'

In [18]:
x_test1=[' '.join(document) for document in x_test]

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

In [20]:
# as selection of 3 feature for huge data is not good choic
count_vec=CountVectorizer(max_features=2000)
x_train_features=count_vec.fit_transform(x_train1)
x_train_features.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [21]:
x_test_features=count_vec.transform(x_test1)

In [22]:
from sklearn.svm import SVC

In [23]:
svc=SVC()
svc.fit(x_train_features,y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [24]:
y_predicted=svc.predict(x_test_features)

In [25]:
y_predicted

array(['negative', 'negative', 'negative', ..., 'neutral', 'positive',
       'negative'], dtype=object)

In [ ]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV #CV stands for cross validation
clf=KNeighborsClassifier()
lt=np.arange(1,26,2)
grid={"n_neighbors":lt}# it takes the values of parameter to be passed
abc=GridSearchCV(clf,grid)
abc.fit(x_train_features,y_train)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
clf=KNeighborsClassifier()
clf.fit(x_train_features,y_train)
y_predicted=clf.predict(x_test_features)

In [ ]:
y_predicted=svc.predict(x_test_features)

In [ ]:
y_predicted

In [ ]:
y_predicted=np.array(y_predicted)

In [ ]:
np.savetxt("twitter_sentiment21.csv",y_predicted,delimiter=',',fmt="%s")